In [45]:
import pandas as pd
from pandasql import sqldf
import boto3
import json
import os
# python -m spacy download en
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# import numpy as np
# from keras.utils import to_categorical
# from keras.models import Sequential
# from keras.layers import Dense
# from sklearn.metrics import confusion_matrix
# import seaborn as sns
# import matplotlib.pyplot as plt

In [ ]:
with open('../boto-config.json') as json_data:
    boto_config = json.load(json_data)
    
s3 = boto3.client(
    's3',
    aws_access_key_id=boto_config['aws_access_key_id'],
    aws_secret_access_key=boto_config['aws_secret_access_key']
)

In [59]:
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
src_file = data_dir + '/News_Category_Dataset.json'
src_file_cleaned = data_dir + '/News_Clean.csv'

# s3.upload_file(src_file, boto_config['buckets']['kaggle'], src_file)
# s3.upload_file(src_file_cleaned, boto_config['buckets']['kaggle'], src_file_cleaned)

# s3.download_file(boto_config['buckets']['kaggle'], src_file, src_file)
s3.download_file(boto_config['buckets']['kaggle'], src_file_cleaned, src_file_cleaned)

In [28]:
data = pd.read_json(src_file, lines=True)
df = pd.read_csv(src_file_cleaned)

In [29]:
data.describe()

,authors,category,date,headline,link,short_description
count,124989,124989,124989,124989,124989,124989
unique,19250,31,1498,124560,124964,103905
top,,POLITICS,2017-01-27 00:00:00,Sunday Roundup,https://www.huffingtonpost.comhttp://highline....,
freq,14151,32739,100,90,2,19590
first,NaN,NaN,2014-04-18 00:00:00,NaN,NaN,NaN
last,NaN,NaN,2018-05-26 00:00:00,NaN,NaN,NaN


In [6]:
data.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


In [37]:
data.isnull().values.any()

False

In [30]:
data.columns

Index(['authors', 'category', 'date', 'headline', 'link', 'short_description'], dtype='object')

In [31]:
data.shape

(124989, 6)

In [51]:
# clean up text
nlp = spacy.load('en')

data['text'] = data['headline'] + ' ' + data['short_description']

docs = data['text'].tolist()

def token_filter(token):
    return not (token.is_punct | token.is_space | token.is_stop | len(token.text) <= 3)

filtered_tokens = []
for doc in nlp.pipe(docs):
    tokens = [token.lemma_ for token in doc if token_filter(token) and token.text not in STOP_WORDS]
    filtered_tokens.append(' '.join(tokens))

data['clean_text'] = filtered_tokens

data.to_csv(src_file_cleaned, index=False)

In [17]:
# tfidf with keras
# nn with keras

In [11]:
# data = pd.read_csv('clean_data.csv')

In [13]:
# encode class values as integers
# encoder = LabelEncoder()
# encoder.fit(data['category'])
# y = encoder.transform(data['category'])

In [14]:
# X_train, X_test, y_train, y_test = train_test_split(data.clean_text, 
#                                                     y, 
#                                                     test_size = 0.2, 
#                                                     random_state = 1, 
#                                                     stratify = y)

# categories = list(np.unique(y))

In [15]:
# tfidf = TfidfVectorizer(ngram_range=(1,1), use_idf=True, analyzer="word")

In [16]:
# X_train = tfidf.fit_transform(X_train.values.astype('U'))
# X_test = tfidf.transform(X_test.values.astype('U'))

# y_train = to_categorical(y_train, num_classes = len(categories))
# y_test = to_categorical(y_test, num_classes = len(categories))

# model = Sequential()
# model.add(Dense(80, input_dim=X_train.shape[1], activation='relu'))
# model.add(Dense(len(categories), activation='sigmoid'))

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])